In [2]:
import torch
import numpy as np
from qmc.tracehess import autograd_trace_hessian
from torch import nn, optim

In [139]:
class NelectronVander(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i*r_i)
    #input is 1D tensor which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha):
        super(NelectronVander, self).__init__()
        self.alpha = nn.Parameter(alpha)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.prod(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    
        

In [140]:
class NelectronVanderWithMult(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i * r_i) 
    #multiplied by e^(-beta * (r_1 + r_2 + ... r_N))
    #input is alpha, beta where alpha is 1D tensor which determines the number of particles and beta is scalar
    def __init__(self, alpha, beta):
        super(NelectronVanderWithMult, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.beta = nn.Parameter(beta)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * ( -self.beta * torch.sum(x, -1)
            + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ])),-1) )
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-self.beta * torch.sum(x, -1)) * torch.prod(a[...,torch.triu(torch.ones(a.shape[-1],a.shape[-1]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(5,5), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    

In [141]:
f = Vander(torch.rand(5))

In [142]:
f(torch.rand((3,5)))

tensor([-51.4964, -69.6246, -44.5690], grad_fn=<MulBackward0>)

In [143]:
f.wave(torch.rand((3,7,5)))

tensor([[ 1.8694e-09, -2.8327e-10,  1.3979e-12, -4.4184e-14, -4.1318e-10,
          7.2100e-10, -7.7997e-13],
        [-2.7059e-09, -1.2367e-12, -5.7399e-11, -8.0265e-12, -2.8806e-14,
         -6.4007e-13,  2.4216e-12],
        [ 1.8610e-13, -6.9472e-11, -5.4461e-11, -1.7440e-10,  3.3216e-10,
          3.2762e-10,  8.8517e-13]], grad_fn=<ProdBackward1>)

In [144]:
x = torch.rand(4,6,5)

In [145]:
f = VanderWithMult(torch.rand(5),torch.rand(1))

In [146]:
f(torch.rand((9,5)))

tensor([-53.4959, -62.1351, -48.9421, -53.1415, -51.5963, -50.9587, -66.0183,
        -51.8630, -53.8266], grad_fn=<MulBackward0>)

In [147]:
f.wave(torch.rand((3,7,5)))

tensor([[-1.1833e-11, -7.2249e-15,  1.1890e-15, -5.4304e-13, -5.0313e-13,
         -4.4071e-12,  1.7126e-11],
        [-2.5826e-16, -9.0713e-13,  6.5776e-14,  6.9954e-11, -6.2504e-12,
         -3.5978e-11,  1.3293e-12],
        [-2.4254e-15, -1.4774e-12,  9.6951e-13, -9.8323e-13,  6.8342e-13,
          8.5196e-12,  1.3433e-10]], grad_fn=<MulBackward0>)